In [ ]:
%% capture

!git clone https://github.com/agoryuno/deepsolo-onnx

import os
import shutil
os.chdir('deepsolo-onnx')

!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt

!python setup.py sdist bdist_wheel
!pip install dist/*.whl

shutil.copy('configs/Base_det_export.yaml', '/content')

os.chdir('/content')

!rm -rf deepsolo-onnx

In [ ]:
# This is for quick reloads during development- DON"T EXECUTE
assert False
!pip install --force-reinstall --no-deps git+https://github.com/agoryuno/deepsolo-onnx

In [ ]:
from DeepSolo.onnx_model import SimpleONNXReadyModel

# Specify the path to the DeepSolo checkpoint you want to export
# you can download one from the main DeepSolo repo: 
# https://github.com/ViTAE-Transformer/DeepSolo
#
CHECKPOINT = "/content/drive/MyDrive/deepsolo/vitaev2-s_pretrain_synth-tt-mlt-13-15-textocr.pth"

# This is the default config file
# you probably should leave it alone
#
CONFIG = "/content/Base_det_export.yaml"

model = SimpleONNXReadyModel(CONFIG, CHECKPOINT)

In [3]:
# Creating a fake "image" to set the input-output dimensions
# of the exported model's data
#
# 

# Input and output image dimensions - these are fixed 
# for the exported ONNX model, so choose wisely
#
DIMS = (512,512)

import torch
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode
from DeepSolo.adet.utils.visualizer import TextVisualizer

import numpy as np

# The number of channels in an image
# This can be left at default
#
CHANNELS = 3

img = np.random.randint(0, 255, (CHANNELS, *DIMS))

In [ ]:
import torch.onnx
import torch

img_t = torch.from_numpy(img).permute(2, 0, 1)

torch.onnx.export(model.model,
           [img_t],
            #[{"image": img_t}],
           "deepsolo.onnx",
           export_params=True,
           do_constant_folding=True,)

In [ ]:
# Everything below requires the default DeepSolo model to work

predictions = model(img)
metadata = MetadataCatalog.get("__unused")
instance_mode = ColorMode.IMAGE
visualizer = TextVisualizer(img, metadata, instance_mode=instance_mode, cfg=model.cfg)
vis_output = None
instances = predictions["instances"].to(torch.device("cpu"))
vis_output = visualizer.draw_instance_predictions(predictions=instances)


In [ ]:
import cv2
import matplotlib.pyplot as plt

out_img = vis_output.get_image()[:, :, ::-1]
plt.imshow(out_img)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
import numpy as np

ctrl_pnts = instances.ctrl_points.numpy()
scores = instances.scores.tolist()
recs = instances.recs
bd_points = np.asarray(instances.bd)

for ctrl_pnt, score, rec, bd in zip(ctrl_pnts, scores, recs, bd_points):
  if bd is not None:
    # the bounding polygon
    bd = np.hsplit(bd, 2)
    bd = np.vstack([bd[0], bd[1][::-1]])

    # decode text
    txt = visualizer._ctc_decode_recognition(rec)
    print (txt)

    # centerline
    line = visualizer._process_ctrl_pnt(ctrl_pnt)
    print (line)

In [ ]:
type(model.model)


DeepSolo.onnx_model.ViTAEPredictor